# EnergyModels

## Minimal 3-node example of linear optimal power flow

In [85]:
]activate .; instantiate

Activating environment at `~/.julia/dev/EnergyModels/examples/Project.toml`


In [86]:
using PowerModels
const PM = PowerModels
using JuMP
using Dates
using AxisArrays
using Gurobi

In [87]:
using Revise
using EnergyModels
const EM = EnergyModels

EnergyModels

### Start new empty EnergyModel for linear dispatch

In [88]:
model = EnergyModel(modeltype=EM.LinearDispatchModel, optimizer=Gurobi.Optimizer)

Academic license - for non-commercial use only


EnergyModel{EnergyModels.LinearDispatchModel,DCPlosslessForm} with 0 buses and 0 devices

In [89]:
set_snapshots!(model, [:now]) # or DateTime("2012-01"):Hour(1):DateTime("2012-01-31 23:00")

Axis{:snapshots,Array{Symbol,1}}(Symbol[:now])

### Adding components

In [90]:
for i in 1:3
    add!(model, Bus, Symbol(:bus, i), carrier=:AC)
end

In [91]:
#add three lines in a ring
for i in 1:3
    add!(model, Line, Symbol(:line, i),
         bus0=Symbol(:bus, i),
         bus1=Symbol(:bus, i%3+1),
         x=0.0001,
         s_nom=60.)
end

In [92]:
add!(model, Generator, :gen1, bus=:bus1, p_nom=100., marginal_cost=50.)
add!(model, Generator, :gen2, bus=:bus2, p_nom=100., marginal_cost=25.)

Generator{EnergyModels.LinearExpansionForm{EnergyModels.LinearDispatchForm}}(gen2)

In [93]:
add!(model, Load, :load, bus=:bus3, p_set=100.)

Load{EnergyModels.LinearDispatchForm}(load)

### Building the JuMP model

In [94]:
determine_subnetworks!(model) # now implicit in `build!(model)`

In [95]:
build!(model)

A JuMP Model
Minimization problem with:
Variables: 5
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 4 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 5 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 5 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi

### Solving it

In [96]:
optimize!(model)

Optimize a model with 4 rows, 5 columns and 11 nonzeros
Coefficient statistics:
  Matrix range     [1e-04, 1e+00]
  Objective range  [2e+01, 5e+01]
  Bounds range     [6e+01, 1e+02]
  RHS range        [1e+02, 1e+02]
Presolve removed 4 rows and 5 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  3.000000000e+03


## Introspection

### Single components

Each component is accessible using `model[<name>]`

In [109]:
g = model[:gen1]

Generator{EnergyModels.LinearExpansionForm{EnergyModels.LinearDispatchForm}}(gen1) (demoted to EnergyModels.LinearDispatchForm)

... the same `[]` syntax on a component gives access to parameters

In [116]:
@show g[:p_nom] g[:marginal_cost];

g[:p_nom] = 100.0
g[:marginal_cost] = 50.0


... as well as variables (time-dependent arrays)

In [113]:
g[:p]

2-dimensional AxisArray{VariableRef,2,...} with axes:
    :gen1, Symbol[:gen1]
    :snapshots, Symbol[:now]
And data, a 1×1 Array{VariableRef,2}:
 p[gen1,now]

and their values

In [114]:
value.(g[:p])

1×1 Array{Float64,2}:
 20.0

... as well as constraints

In [119]:
model[:bus1][:p_balance]

2-dimensional AxisArray{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,2,...} with axes:
    :bus1, Symbol[:bus1]
    :snapshots, Symbol[:now]
And data, a 1×1 Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,2}:
 p_balance[bus1,now] : p[gen1,now] - p[line3,now] + p[line1,now] = 0.0

In [120]:
dual.(model[:bus1][:p_balance])

1×1 Array{Float64,2}:
 50.0

In [121]:
value.(model[:bus1][:p_balance])

1×1 Array{Float64,2}:
 0.0

### ContainerViews

Instead of single components, we can also query across multiple components with the concrete or even abstract types

... capacity of all `Generator`s:

In [126]:
model[Generator][:p_nom]

1-dimensional AxisArray{Float64,1,...} with axes:
    :generators, Tuple{Symbol,Symbol}[(:gen2, :gen2), (:gen1, :gen1)]
And data, a 2-element Array{Float64,1}:
 100.0
 100.0

... dispatch of all `Branch`es

In [141]:
model[Branch][:p]

2-dimensional AxisArray{VariableRef,2,...} with axes:
    :branches, Tuple{Symbol,Symbol}[(:line2, :line2), (:line3, :line3), (:line1, :line1)]
    :snapshots, Symbol[:now]
And data, a 3×1 Array{VariableRef,2}:
 p[line2,now]
 p[line3,now]
 p[line1,now]

In [133]:
value.(model[Line][:p])

3×1 Array{Float64,2}:
 -60.0
  40.0
  20.0

### Duals of the line-flow variable bounds (ie how binding are line capacities)

In [137]:
dual.(UpperBoundRef.(model[Line][:p]))

3×1 Array{Float64,2}:
 0.0
 0.0
 0.0

In [135]:
dual.(LowerBoundRef.(model[Line][:p]))

3×1 Array{Float64,2}:
 75.0
  0.0
  0.0

# From PyPSA: linear optimal power flow

In [1]:
import pypsa
import numpy as np

In [2]:
network = pypsa.Network()

In [3]:
#add three buses
for i in range(3):
    network.add("Bus","My bus {}".format(i))

In [4]:
#add three lines in a ring
for i in range(3):
    network.add("Line","My line {}".format(i),
                bus0="My bus {}".format(i),
                bus1="My bus {}".format((i+1)%3),
                x=0.0001,
                s_nom=60)

In [5]:
#add a generator at bus 0
network.add("Generator","My gen 0",
            carrier="gas",
            bus="My bus 0",
            p_nom=100,
            marginal_cost=50)

#add a generator at bus 1
network.add("Generator","My gen 1",
            carrier="gas",
            bus="My bus 1",
            p_nom=100,
            marginal_cost=25)

In [6]:
#add a load at bus 2
network.add("Load","My load",
            bus="My bus 2",
            p_set=100)

## EnergyModels in PyPSA

We import Julia's EnergyModels and Gurobi from the (local) environment

In [7]:
from julia import Pkg
Pkg.activate(".")
from julia import EnergyModels as EM, Gurobi

.. and are able to mix PyPSA and EnergyModels freely: Data is taken straight out of the pandas dataframes, only short-lived copies during the building phase of each component:

In [8]:
model = EM.EnergyModel(network, optimizer=Gurobi.Optimizer)
EM.build_b(model)

<PyCall.jlwrap Min 50 gas::p[My gen 0,now] + 25 gas::p[My gen 1,now]
Subject to
 gas::p[My gen 0,now] + gas::p[My gen 1,now] = 100.0
 gas::p[My gen 0,now] + gas::p[My gen 1,now] = 100.0
 gas::p[My gen 0,now] + gas::p[My gen 1,now] = 100.0
 -0.0001 lines_AC::p[My line 0,now] - 0.0001 lines_AC::p[My line 1,now] - 0.0001 lines_AC::p[My line 2,now] = 0.0
 lines_AC::p[My line 0,now] ≥ -60.0
 lines_AC::p[My line 1,now] ≥ -60.0
 lines_AC::p[My line 2,now] ≥ -60.0
 gas::p[My gen 0,now] ≥ 0.0
 gas::p[My gen 1,now] ≥ 0.0
 lines_AC::p[My line 0,now] ≤ 60.0
 lines_AC::p[My line 1,now] ≤ 60.0
 lines_AC::p[My line 2,now] ≤ 60.0
 gas::p[My gen 0,now] ≤ 100.0
 gas::p[My gen 1,now] ≤ 100.0
>

... and optimize

In [9]:
EM.optimize_b(model)

... and write back

In [ ]:
EM.store_results_b(model)

In [21]:
#Cheap generator 1 cannot be fully dispatched because of network constraints,
#so expensive generator 0 also has to dispatch
print(network.generators_t.p)

generators_i  My gen 0  My gen 1
snapshots                       
now               20.0      80.0


In [22]:
#network flows
print(network.lines_t.p0)

lines_i    My line 0  My line 1  My line 2
snapshots                                 
now            -20.0       60.0      -40.0


In [23]:
#Line 1 is congested
print(abs(network.lines_t.p0)/network.lines.s_nom)

lines_i    My line 0  My line 1  My line 2
snapshots                                 
now         0.333333        1.0   0.666667


In [24]:
#Power flows towards lower voltage angles
print(network.buses_t.v_ang*180/np.pi)

buses_i    My bus 0  My bus 1  My bus 2
snapshots                              
now             0.0  0.114592 -0.229183


In [25]:
#In linear approximation, all voltage magnitudes are nominal, i.e. 1 per unit
print(network.buses_t.v_mag_pu)

buses_i    My bus 0  My bus 1  My bus 2
snapshots                              
now             1.0       1.0       1.0


In [26]:
#At bus 2 the price is set above any marginal generation costs in the model, because to dispatch to
#it from expensive generator 0, also some dispatch from cheap generator 1 has to be substituted from generator0
#to avoid overloading line 1.
print(network.buses_t.marginal_price)

buses_i    My bus 0  My bus 1  My bus 2
snapshots                              
now            50.0      25.0      75.0


# Alternative `lopf` based on EnergyModels

In [34]:
def lopf_with_energymodels(network, snapshots=None):
    model = EM.EnergyModel(network, optimizer=Gurobi.Optimizer)
    if snapshots is not None:
        EM.set_snapshots_b(snapshots)
    EM.build_b(model)
    EM.optimize_b(model)
    EM.store_results_b(model)

... and go

In [ ]:
lopf_with_energymodels(network)

## Transforming into an expansion model

In [143]:
exp = EnergyModel(model, modeltype=EM.ExpansionModel, optimizer=Gurobi.Optimizer)

Academic license - for non-commercial use only


EnergyModel{EnergyModels.ExpansionModel,DCPlosslessForm} with 3 buses and 6 devices

In [146]:
build!(exp)

A JuMP Model
Minimization problem with:
Variables: 10
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 4 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 5 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 5 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 5 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 5 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi

## Formulations

In [ ]:
# Dispatch formulations
abstract type DispatchForm <: DeviceFormulation end

struct LinearDispatchForm <: DispatchForm end
struct UnitCommittmentForm <: DispatchForm end

# Expansion Formulations
abstract type ExpansionForm{T<:DispatchForm} <: DeviceFormulation end
struct LinearExpansionForm{T<:DispatchForm} <: ExpansionForm{T} end
struct LinearExpansionDispatchForm{T<:DispatchForm} <: ExpansionForm{T} end
struct LinearExpansionInvestmentForm{T<:DispatchForm} <: ExpansionForm{T} end

In [153]:
model.devices

Dict{Symbol,Device} with 6 entries:
  :gen2  => Generator{LinearExpansionForm{LinearDispatchForm}}(gen2)
  :gen1  => Generator{LinearExpansionForm{LinearDispatchForm}}(gen1)
  :line2 => Line{LinearExpansionForm{LinearDispatchForm}}(line2)
  :line3 => Line{LinearExpansionForm{LinearDispatchForm}}(line3)
  :load  => Load{LinearDispatchForm}(load)
  :line1 => Line{LinearExpansionForm{LinearDispatchForm}}(line1)

## Source structure w/ focus on components

In [163]:
;tree ~/.julia/dev/EnergyModels/src

/home/coroa/.julia/dev/EnergyModels/src
├── abstracttypes.jl
├── compat.jl
├── components
│   ├── attrs
│   │   ├── buses.csv
│   │   ├── generators.csv
│   │   ├── lines.csv
│   │   ├── links.csv
│   │   ├── loads.csv
│   │   ├── storageunits.csv
│   │   ├── stores.csv
│   │   └── transformers.csv
│   ├── bus.jl
│   ├── components.jl
│   ├── energymodel.jl
│   ├── generator.jl
│   ├── link.jl
│   ├── load.jl
│   ├── passivebranch.jl
│   ├── storageunit.jl
│   ├── store.jl
│   └── subnetwork.jl
├── containerviews.jl
├── core.jl
├── data
│   ├── data.jl
│   ├── powersystems.jl
│   ├── pypsa.devices.csv
│   ├── pypsa.jl
│   ├── pypsa.lines.types.csv
│   ├── pypsanetwork.jl
│   └── pypsa.transformers.types.csv
├── EnergyModels.jl
├── formulation.jl
├── graph.jl
├── macros.jl
├── modelview.jl
├── powermodels.jl
├── registry.jl
└── wrappedarray.jl

3 directories, 37 files


In [166]:
EM.set_formulation!(model, Line=>EM.LinearExpansionForm{EM.LinearDispatchForm})

In [167]:
model.devices

Dict{Symbol,Device} with 6 entries:
  :gen2  => Generator{LinearExpansionForm{LinearDispatchForm}}(gen2)
  :gen1  => Generator{LinearExpansionForm{LinearDispatchForm}}(gen1)
  :line2 => Line{LinearExpansionForm{LinearDispatchForm}}(line2)
  :line3 => Line{LinearExpansionForm{LinearDispatchForm}}(line3)
  :load  => Load{LinearDispatchForm}(load)
  :line1 => Line{LinearExpansionForm{LinearDispatchForm}}(line1)

In [ ]:
push!(model.data, :line2, )

In [ ]:
exp = EnergyModel(model, modeltype=)

In [168]:
model.data.components[:onwind][:p_nom] = AxisArray{}

EnergyModels.Data{Nothing}(Dict(:gen1 => EnergyModels.ComponentDesc(:gen1, Generator, Dict{Symbol,Union{Bool, Float64, Int64, Symbol, AxisArray}}(:p_nom => 100.0,:bus => :bus1,:marginal_cost => 50.0)),:gen2 => EnergyModels.ComponentDesc(:gen2, Generator, Dict{Symbol,Union{Bool, Float64, Int64, Symbol, AxisArray}}(:p_nom => 100.0,:bus => :bus2,:marginal_cost => 25.0)),:bus1 => EnergyModels.ComponentDesc(:bus1, Bus, Dict{Symbol,Union{Bool, Float64, Int64, Symbol, AxisArray}}(:carrier => :AC)),:bus3 => EnergyModels.ComponentDesc(:bus3, Bus, Dict{Symbol,Union{Bool, Float64, Int64, Symbol, AxisArray}}(:carrier => :AC)),:line2 => EnergyModels.ComponentDesc(:line2, Line, Dict{Symbol,Union{Bool, Float64, Int64, Symbol, AxisArray}}(:bus1 => :bus3,:s_nom => 60.0,:bus0 => :bus2,:x => 0.0001)),:line3 => EnergyModels.ComponentDesc(:line3, Line, Dict{Symbol,Union{Bool, Float64, Int64, Symbol, AxisArray}}(:bus1 => :bus1,:s_nom => 60.0,:bus0 => :bus3,:x => 0.0001)),:load => EnergyModels.ComponentDesc(

In [169]:
model.jumpobjects

Dict{Symbol,Dict{Symbol,Any}} with 10 entries:
  :gen2        => Dict{Symbol,Any}(:p_upper=>2-dimensional DenseAxisArray{Const…
  :line1       => Dict{Symbol,Any}(:p_upper=>2-dimensional DenseAxisArray{Const…
  :bus1        => Dict{Symbol,Any}(:p_balance=>2-dimensional DenseAxisArray{Con…
  :line2       => Dict{Symbol,Any}(:p_upper=>2-dimensional DenseAxisArray{Const…
  :bus3        => Dict{Symbol,Any}(:p_balance=>2-dimensional DenseAxisArray{Con…
  :line3       => Dict{Symbol,Any}(:p_upper=>2-dimensional DenseAxisArray{Const…
  :load        => Dict{Symbol,Any}()
  :subnetwork1 => Dict{Symbol,Any}(:cycles=>2-dimensional DenseAxisArray{Constr…
  :bus2        => Dict{Symbol,Any}(:p_balance=>2-dimensional DenseAxisArray{Con…
  :gen1        => Dict{Symbol,Any}(:p_upper=>2-dimensional DenseAxisArray{Const…

In [ ]:
model.data.fallback